In [2]:
%load_ext autoreload
%autoreload 2
from functools import partial

import keras

from datasets import example_datasets, to_numpy
from models import mixture_poissons,poisson_glm
from metrics import mixture_poi_loss, get_bpr_loss_func, mix_bpr, get_penalized_bpr_loss_func_mix, cross_ratio_decision
from experiments import training_loop, training_loop_score_function_trick
from plotting_funcs import plot_losses, plot_frontier

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

2024-05-11 08:28:55.261051: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-11 08:28:56.898674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 08:28:56.898761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 08:28:57.431313: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 08:28:58.169163: I tensorflow/core/platform/cpu_feature_guar

In [3]:
seed=360
num_components=4
learning_rate = 0.005
epochs=500
outdir = '/cluster/home/kheuto01/testdir'
penalty = 5000
threshold = 0.55
K=4
do_only=True
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 500
perturbed_sigma=0.1

In [4]:
train_dataset, val_dataset, test_dataset = example_datasets(H, T, seed=seed)
train_X_THS, train_y_TS = to_numpy(train_dataset)
val_X_THS, val_y_TS = to_numpy(val_dataset)
input_shape = (H,S)

negative_bpr_K = get_bpr_loss_func(K)

In [5]:
# Penalized loss
mix_model_penalized, _  = mixture_poissons(poisson_glm, input_shape, num_components=num_components)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
penalized_bpr_loss = get_penalized_bpr_loss_func_mix(mixture_poi_loss, K, penalty, threshold)

In [6]:
    train_X_THS, train_y_TS = to_numpy(train_dataset)
    val_X_THS, val_y_TS = to_numpy(val_dataset)

In [7]:
model=mix_model_penalized

In [8]:
for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    with tf.GradientTape() as jacobian_tape, tf.GradientTape() as loss_tape:
                prob_params_BSK, mixture_weights_KS = model(x_batch_train, training=True)

In [9]:
mixture_weights_SK = tf.transpose(mixture_weights_KS, perm=[1,0])

In [10]:
cat = tfp.distributions.Categorical(probs=mixture_weights_SK)

In [11]:
cat.sample(2)

<tf.Tensor: shape=(2, 12), dtype=int32, numpy=
array([[2, 0, 1, 0, 1, 2, 1, 3, 3, 1, 0, 0],
       [3, 0, 0, 1, 3, 3, 3, 2, 3, 2, 3, 3]], dtype=int32)>

In [10]:
losses_penalized = training_loop_score_function_trick(mix_model_penalized, penalized_bpr_loss, optimizer,
                                    epochs, train_dataset, val_dataset, negative_bpr_K,
                                    cross_ratio_decision,
                                    num_score_func_samples=10,
                                    component_likelihood=tfp.distributions.Poisson,
                                    verbose=True)

OperatorNotAllowedInGraphError: in user code:

    File "/cluster/home/kheuto01/code/prob_diff_topk/metrics.py", line 150, in uncurried_penalized_mix_bpr  *
        mixture_bpr_loss_val = mix_bpr(y_true, y_pred, negative_bpr_K_func=negative_bpr_K_func)
    File "/cluster/home/kheuto01/code/prob_diff_topk/metrics.py", line 138, in mix_bpr  *
        component_preds, mixture_weights = y_pred

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more in